In [1]:
import collections
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data = {}
for i in range(25):
    data[f'df{i}'] = pd.read_csv(f'~/git_env/research/oshea/trident_modifications/testing_mods/SolAb/test_sal/abundance_test3/data/data_row{i}_.csv', delim_whitespace=True)

In [3]:
df1 = data['df0']

def what_ions(df, ion_list, ray_index):
    """
    determines what ions are returned in handy dandy dataframe from salsa 
    
    :ion_list: list of ions initially passed to salsa
    
    :ray: lightray index from salsa dataframe
    """
    names = [i for i in ion_list if i in df['name'].tolist() and i in ray_index['name'].tolist()]

    return names

names = what_ions(df = df1, ion_list = ['Ne VIII', 'Mg X', 'O VI', 'S IV', 'Si III', 'C II', 'N I'], ray_index = df1[df1['lightray_index'] == 62])


In [4]:
def pick_intervals(file_dictionary, lightray):
    
    master_df = pd.DataFrame()
    
    for df in file_dictionary:
        obj = file_dictionary[df]
        master_df = master_df.append(obj[obj['lightray_index'] == lightray])
        
    master_df.reset_index(inplace=True)
    
    index_list = []
    
    for index in range(len(master_df)):
        
        magic_start = master_df['interval_start'][index]
        magic_end = master_df['interval_end'][index]

        if index == 0:
            index_list.append([magic_start, magic_end])
        elif [magic_start, magic_end] in index_list:
            pass
        else:
            
            count = 0

            for x in range(len(index_list)):

                if [magic_start, magic_end] in index_list:
                    pass
                
                elif index_list[x][0] == magic_start:
                    if magic_end > index_list[x][1]:
                        index_list[x][1] = magic_end
                    else:
                        count += 1

                elif index_list[x][1] == magic_end:
                    if magic_start < index_list[x][0]:
                        index_list[x][0] = magic_start
                    else:
                        count += 1
  
            if [magic_start, magic_end] not in index_list and count == 0:
                index_list.append([magic_start, magic_end])
            
    return master_df, index_list

In [5]:
master_df, index_list = pick_intervals(data, 62)
index_list

[[102, 132.0],
 [178, 215.0],
 [373, 384.0],
 [351, 365.0],
 [215, 219.0],
 [232, 357.0],
 [258, 283.0],
 [284, 357.0],
 [357, 365.0],
 [234, 264.0],
 [266, 270.0],
 [273, 276.0],
 [277, 279.0]]

In [6]:
for interval in index_list:
    df = master_df[(master_df['interval_start'] >= interval[0]) & (master_df['interval_end'] <= interval[1])]
    name_list=[]
    for name in df['name']:
        name_list.append(name)
    print(f"interval: {interval}, {collections.Counter(name_list)}")
    print(f"df length = {len(df)}")
    df.reset_index(inplace=True)
    plt.figure()
    hist_plot(df, 'col_dens')
    plt.show()
    

interval: [102, 132.0], Counter({'O VI': 25})
df length = 25


NameError: name 'hist_plot' is not defined

<Figure size 432x288 with 0 Axes>

In [ ]:
test_df = master_df[(master_df['interval_start'] >= 100) & (master_df['interval_end'] <= 150)]
name_list = []
for name in test_df['name']:
    name_list.append(name)

print(collections.Counter(name_list))
test_df.reset_index(inplace=True)

test_df

In [ ]:
def hist_plot(absorber_df, thing_we_care_about, xleft=13.0, xright=13.6, lims=False):
    
    dic = {}
    for index in range(len(absorber_df)):
        if index == 0:
            dic[f"{absorber_df['name'][index]}"] = []
        else:
            if absorber_df['name'][index] == absorber_df['name'][index-1]:
                pass
            else:
                dic[f"{absorber_df['name'][index]}"] = []
                
    for ion in dic:
        for index in range(len(absorber_df)):
            if absorber_df['name'][index] == ion:
                dic[ion].append(absorber_df[thing_we_care_about][index])
#         print(dic[ion])
        plt.hist(dic[ion], label=ion)
        if lims is not False:
            plt.xlim(xleft, xright)
    plt.legend()
    
    return dic

In [ ]:
dic=hist_plot(test_df, 'col_dens')

In [ ]:
exab_df